1. 초기 설정 및 라이브러리 임포트

In [1]:
# 필요한 라이브러리 임포트
import os
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
from tqdm.notebook import tqdm  # 주피터 노트북에 최적화된 진행 표시줄
import concurrent.futures  # 병렬 처리용
import logging

# 경고 메시지 설정
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)

# 로깅 설정
logging.basicConfig(level=logging.INFO, 
                   format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# 기본 경로 설정
BASE_PATH = Path(r"C:\Users\markcloud\Desktop\오준호\IBK")

2. 유틸리티 함수 정의

In [2]:
# 파일 인코딩 자동 감지 함수
def detect_encoding(file_path):
    """파일의 인코딩을 자동으로 감지하는 함수"""
    encodings = ['utf-8', 'euc-kr', 'cp949', 'utf-16']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                f.read(100)  # 일부만 읽어서 테스트
                return encoding
        except UnicodeDecodeError:
            continue
    return None

# 텍스트 파일 안전하게 읽기 함수
def read_text_file_safely(file_path, default_value=""):
    """텍스트 파일을 안전하게 읽는 함수"""
    if not os.path.exists(file_path):
        return default_value
    
    encoding = detect_encoding(file_path)
    if not encoding:
        logger.warning(f"적합한 인코딩을 찾을 수 없음: {file_path}")
        return default_value
    
    try:
        with open(file_path, 'r', encoding=encoding) as f:  # 여기서 errors='replace' 매개변수가 없는지 확인
            return f.read()
    except Exception as e:
        logger.warning(f"파일 읽기 오류: {file_path} - {e}")
        return default_value

# OCR 텍스트 파일 읽기 함수 (병렬 처리에 최적화)
def read_ocr_file(args):
    """OCR 파일을 읽는 함수 (병렬 처리용)"""
    subdir, ocr_file = args
    if not ocr_file or pd.isna(ocr_file):
        return ""
    
    ocr_file_path = os.path.join(subdir, ocr_file)
    return read_text_file_safely(ocr_file_path)

# 폴더 번호에 따른 상위 폴더명 반환 함수
def get_correct_base_folder(num):
    """폴더 번호에 따른 상위 폴더명 반환 함수"""
    try:
        num = int(str(num).strip())
        if 24801 <= num <= 28103:
            return "ibk_24801_start_2"
        elif 28104 <= num <= 32080:
            return "ibk_28104_start_2"
        elif 32081 <= num <= 41902:
            return "ibk_32081_start_2"
        else:
            return None
    except (ValueError, TypeError):
        return None

3. OCR 데이터 수집 및 처리 함수

In [3]:
def collect_csv_data(folder_path):
    """CSV 파일 데이터를 수집하는 함수"""
    all_csv_files = []
    folder_path = Path(folder_path)
    
    try:
        # 모든 하위 폴더 탐색
        for subdir in tqdm(list(folder_path.glob('**/')), desc="폴더 스캔 중"):
            subdir_str = str(subdir)
            csv_files = list(subdir.glob('*.csv'))
            
            if csv_files:
                # CSV 파일 처리
                for csv_file in csv_files:
                    try:
                        # 수정된 부분: errors 매개변수 제거하고 인코딩 처리 개선
                        try:
                            df = pd.read_csv(csv_file, encoding='utf-8')
                            logger.info(f"CSV 파일 읽기 성공: {csv_file} ({len(df)} 행)")
                        except UnicodeDecodeError:
                            # 다른 인코딩 시도
                            df = pd.read_csv(csv_file, encoding='cp949')
                            logger.info(f"CSV 파일 읽기 성공(cp949): {csv_file} ({len(df)} 행)")
                        
                        # 데이터프레임에 폴더 정보 추가
                        if 'folder' not in df.columns:
                            df['folder'] = subdir.name
                        
                        all_csv_files.append(df)
                    except pd.errors.EmptyDataError:
                        logger.warning(f"빈 CSV 파일: {csv_file}")
                    except Exception as e:
                        logger.error(f"CSV 처리 오류: {csv_file} - {e}")
            else:
                # 이미지 폴더 처리
                images_dir = subdir / 'images'
                if images_dir.exists():
                    image_files = [f.name for f in images_dir.glob('*') if f.is_file()]
                    if image_files:
                        temp_df = pd.DataFrame({
                            'folder': subdir.name,
                            'image': image_files,
                            'ocr_file': '',
                            'processing_time': '',
                            'char_count': '',
                            'ocr_text': ''
                        })
                        all_csv_files.append(temp_df)
        
        if not all_csv_files:
            logger.warning("수집된 CSV 파일이 없습니다!")
            return pd.DataFrame()  # 빈 데이터프레임 반환
        
        # 모든 데이터프레임 병합
        merged_df = pd.concat(all_csv_files, ignore_index=True)
        logger.info(f"총 {len(merged_df)}개 행 수집 완료")
        return merged_df
    
    except Exception as e:
        logger.error(f"CSV 데이터 수집 중 심각한 오류 발생: {e}")
        return pd.DataFrame()  # 오류 발생해도 빈 데이터프레임 반환

def add_ocr_text_parallel(df, max_workers=4):
    """병렬 처리를 통해 OCR 텍스트 내용을 추가하는 함수"""
    if len(df) == 0:
        return df
    
    if 'ocr_file' not in df.columns:
        df['ocr_text'] = ""
        return df
    
    subdirs = df.get('subdir', df['folder'].apply(lambda x: ""))
    args_list = [(subdir, ocr_file) for subdir, ocr_file in zip(subdirs, df['ocr_file'])]
    
    # 진행 상황 표시를 위한 설정
    total = len(args_list)
    logger.info(f"OCR 텍스트 {total}개 로딩 중...")
    
    # 병렬 처리
    ocr_texts = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        ocr_texts = list(tqdm(executor.map(read_ocr_file, args_list), total=total, desc="OCR 텍스트 읽기"))
    
    df['ocr_text'] = ocr_texts
    return df

4. 데이터 타입 처리 및 저장 함수

In [4]:
def clean_and_convert_data_types(df):
    """데이터 타입을 정리하고 변환하는 함수"""
    if len(df) == 0:
        return df
    
    # folder 열을 문자열로 변환
    df['folder'] = df['folder'].astype(str)
    
    # 숫자형 열 처리
    numeric_columns = ['processing_time', 'char_count']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col].replace('', np.nan), errors='coerce')
    
    # object 타입 열을 모두 문자열로 변환 (Parquet 저장 오류 방지)
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].fillna("").astype(str)
    
    return df

def save_to_parquet_safely(df, file_path):
    """데이터프레임을 안전하게 Parquet으로 저장하는 함수"""
    try:
        # 데이터 타입 정제
        df = clean_and_convert_data_types(df)
        
        # 저장
        df.to_parquet(file_path, engine='pyarrow', index=False)
        logger.info(f"Parquet 저장 성공: {file_path} ({len(df)} 행)")
        return True
    except Exception as e:
        logger.error(f"Parquet 저장 오류: {e}")
        
        # 대체 저장 방식 (CSV)
        try:
            csv_path = str(file_path).replace('.parquet', '_backup.csv')
            df.to_csv(csv_path, index=False)
            logger.info(f"CSV로 대체 저장 성공: {csv_path}")
        except Exception as csv_e:
            logger.error(f"CSV 저장도 실패: {csv_e}")
        
        return False

5. 텍스트 통합 및 보강 함수

In [5]:
def process_text_files(grouped_df, base_folder_path):
    """text.txt 파일 처리를 통해 데이터를 보강하는 함수"""
    text_contents = []
    base_folder_path = Path(base_folder_path)
    
    for index, row in tqdm(grouped_df.iterrows(), total=len(grouped_df), desc="텍스트 파일 처리"):
        folder_name = row['folder']
        upper_folder = get_correct_base_folder(folder_name)
        
        if upper_folder is None:
            logger.warning(f"[스킵] 번호 범위 밖: {folder_name}")
            text_contents.append("")
            continue
        
        folder_path = base_folder_path / upper_folder / str(folder_name)
        text_file_path = folder_path / 'text.txt'
        
        # 텍스트 파일 읽기
        text_content = read_text_file_safely(text_file_path)
        text_contents.append(text_content)
    
    # 결과 반영
    grouped_df['text'] = text_contents
    return grouped_df

6. 전체 워크플로우 실행 함수

In [6]:
def run_full_workflow(base_path, output_base_name="IBK_processed_data"):
    """전체 데이터 처리 워크플로우를 실행하는 함수"""
    try:
        # 1. CSV 데이터 수집
        logger.info("===== 1단계: CSV 데이터 수집 =====")
        merged_df = collect_csv_data(base_path)
        
        # None 체크는 필요 없지만, 빈 데이터프레임 체크는 필요
        if merged_df is None or merged_df.empty:
            logger.error("데이터 수집 실패! 처리를 중단합니다.")
            return pd.DataFrame()  # 빈 데이터프레임 반환
        
        # 2. OCR 텍스트 추가
        logger.info("===== 2단계: OCR 텍스트 추가 =====")
        merged_df = add_ocr_text_parallel(merged_df)
        
        # 3. 첫 번째 Parquet 저장
        first_parquet_path = base_path / f"{output_base_name}_step1.parquet"
        logger.info(f"===== 3단계: 첫 번째 Parquet 저장 =====")
        if not save_to_parquet_safely(merged_df, first_parquet_path):
            # 저장 실패 시 파이프라인 계속 진행
            logger.warning("첫 번째 Parquet 저장 실패했지만 계속 진행합니다.")
        
        # 4. 필요한 열만 선택
        logger.info("===== 4단계: 데이터 필터링 =====")
        filtered_df = merged_df[['folder', 'ocr_text']]
        
        # 5. 폴더별 텍스트 통합
        logger.info("===== 5단계: 폴더별 텍스트 통합 =====")
        grouped_df = (
            filtered_df
            .groupby('folder', as_index=False)
            .agg(ocr_text_sum=('ocr_text', lambda x: ' | '.join([str(i) for i in x if i])))
        )
        
        # 6. URL 매핑 추가 (파일이 있을 경우)
        logger.info("===== 6단계: URL 매핑 추가 =====")
        url_mapping_path = base_path.parent / "IBK_문서" / "url_index_mapping.csv"
        if url_mapping_path.exists():
            try:
                df_url_index = pd.read_csv(url_mapping_path)
                logger.info(f"URL 매핑 로드 성공: {len(df_url_index)}개 항목")
                # 여기서 매핑 처리 로직을 추가할 수 있음
            except Exception as e:
                logger.warning(f"URL 매핑 로드 실패: {e}")
        else:
            logger.warning(f"URL 매핑 파일 없음: {url_mapping_path}")
        
        # 7. text.txt 파일 내용 추가
        logger.info("===== 7단계: 텍스트 파일 내용 추가 =====")
        final_df = process_text_files(grouped_df, base_path)
        
        # 8. 최종 결과 저장
        logger.info("===== 8단계: 최종 결과 저장 =====")
        final_parquet_path = base_path / f"{output_base_name}_final.parquet"
        save_to_parquet_safely(final_df, final_parquet_path)
        
        logger.info("===== 전체 워크플로우 완료 =====")
        return final_df
    
    except Exception as e:
        logger.error(f"워크플로우 실행 중 심각한 오류 발생: {e}")
        import traceback
        logger.error(traceback.format_exc())  # 상세 오류 스택 출력
        return pd.DataFrame()  # 오류 발생해도 빈 데이터프레임 반환

7. 실행 코드

In [7]:
# 전체 파이프라인 실행
result_df = run_full_workflow(BASE_PATH, "IBK_24801_32081_add_text_plus")

# 결과 확인
if result_df is not None:
    display(result_df.head())
    print(f"총 {len(result_df)}개 항목 처리됨")
    
    # 데이터 품질 체크
    print("\n== 데이터 결측치 현황 ==")
    display(result_df.isnull().sum())
    
    # 요약 통계
    print("\n== 텍스트 길이 통계 ==")
    result_df['ocr_text_length'] = result_df['ocr_text_sum'].str.len()
    result_df['text_length'] = result_df['text'].str.len()
    display(result_df[['ocr_text_length', 'text_length']].describe())

2025-05-14 13:24:54,760 - INFO - ===== 1단계: CSV 데이터 수집 =====


폴더 스캔 중:   0%|          | 0/59600 [00:00<?, ?it/s]

2025-05-14 13:25:01,352 - INFO - CSV 파일 읽기 성공: C:\Users\markcloud\Desktop\오준호\IBK\ibk_준법감시_유효기간_20250514_131538.csv (420 행)
2025-05-14 13:25:01,402 - INFO - CSV 파일 읽기 성공: C:\Users\markcloud\Desktop\오준호\IBK\ibk_24801_start_2\24801\ocr_results.csv (5 행)
2025-05-14 13:25:01,415 - INFO - CSV 파일 읽기 성공: C:\Users\markcloud\Desktop\오준호\IBK\ibk_24801_start_2\24802\ocr_results.csv (16 행)
2025-05-14 13:25:01,427 - INFO - CSV 파일 읽기 성공: C:\Users\markcloud\Desktop\오준호\IBK\ibk_24801_start_2\24803\ocr_results.csv (16 행)
2025-05-14 13:25:01,439 - INFO - CSV 파일 읽기 성공: C:\Users\markcloud\Desktop\오준호\IBK\ibk_24801_start_2\24804\ocr_results.csv (16 행)
2025-05-14 13:25:01,452 - INFO - CSV 파일 읽기 성공: C:\Users\markcloud\Desktop\오준호\IBK\ibk_24801_start_2\24805\ocr_results.csv (9 행)
2025-05-14 13:25:01,465 - INFO - CSV 파일 읽기 성공: C:\Users\markcloud\Desktop\오준호\IBK\ibk_24801_start_2\24806\ocr_results.csv (14 행)
2025-05-14 13:25:01,476 - INFO - CSV 파일 읽기 성공: C:\Users\markcloud\Desktop\오준호\IBK\ibk_24801_start_2\2480

OCR 텍스트 읽기:   0%|          | 0/442517 [00:00<?, ?it/s]

2025-05-14 13:28:57,557 - INFO - ===== 3단계: 첫 번째 Parquet 저장 =====
2025-05-14 13:29:00,582 - INFO - Parquet 저장 성공: C:\Users\markcloud\Desktop\오준호\IBK\IBK_24801_32081_add_text_plus_step1.parquet (442517 행)
2025-05-14 13:29:00,582 - INFO - ===== 4단계: 데이터 필터링 =====
2025-05-14 13:29:00,592 - INFO - ===== 5단계: 폴더별 텍스트 통합 =====
2025-05-14 13:29:00,903 - INFO - ===== 6단계: URL 매핑 추가 =====
2025-05-14 13:29:00,904 - WARNING - URL 매핑 파일 없음: C:\Users\markcloud\Desktop\오준호\IBK_문서\url_index_mapping.csv
2025-05-14 13:29:00,904 - INFO - ===== 7단계: 텍스트 파일 내용 추가 =====


...

,folder,ocr_text_sum,text
0,24801,,"오창 24시 해장국\n"" 노걸대 감자탕 ""\n오창점\n「 글 · 사진 」\n유감독\..."
1,24802,,엄지척 절로 나오는 여수 이순신광장 빵집\n갓버터도나스\n\n올해로 벌써 여수 여행...
2,24803,,안녕하세요\n포로롱 입니다\n\n오늘은\n대구 교동에 레트로감성이 물씬 느껴지는\n...
3,24804,,[대구 중구] 교동뭉티기\n📍주소: 대구 중구 국채보상로123길 15 102호\n🕰...
4,24805,,남편은 일산에서 꽤 오래 살았다.\n하지만 인천에 오래 살아도 타 지역 사람보다 맛...


총 15111개 항목 처리됨

== 데이터 결측치 현황 ==


folder          0
ocr_text_sum    0
text            0
dtype: int64


== 텍스트 길이 통계 ==


,ocr_text_length,text_length
count,15111.0,15111.000000
mean,0.0,2202.839389
std,0.0,2838.972034
min,0.0,0.000000
25%,0.0,1058.000000
50%,0.0,1636.000000
75%,0.0,2424.500000
max,0.0,76100.000000
